In [2]:
# %reset

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

%matplotlib inline

In [4]:
data = pd.read_csv('csv/animal_train.csv')
data

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
...,...,...,...,...,...,...,...,...,...,...
26724,A702446,NaN,2015-05-14 11:56:00,Transfer,Partner,Cat,Intact Male,1 month,Domestic Shorthair Mix,Brown Tabby/White
26725,A718934,NaN,2016-01-20 18:59:00,Transfer,SCRP,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Brown Tabby
26726,A698128,Zeus,2015-03-09 13:33:00,Adoption,NaN,Dog,Neutered Male,4 years,Old English Bulldog Mix,White/Tan
26727,A677478,NaN,2014-04-27 12:22:00,Transfer,Partner,Cat,Intact Male,4 weeks,Domestic Shorthair Mix,Black


# 1. Обработка

1. Обработка пропущенных значений: В столбцах Name и OutcomeSubtype много пропущенных значений. Вы можете решить удалить столбец OutcomeSubtype, так как он может быть не очень важен для предсказания основного результата. Для столбца Name вы можете создать новую бинарную переменную, указывающую, есть ли имя у животного (1) или нет (0), поскольку наличие имени может влиять на исход приюта. Пропущенные значения в SexuponOutcome и AgeuponOutcome можно заполнить наиболее частыми значениями или средними, в зависимости от контекста.

2. Преобразование типов данных: Преобразуйте столбец DateTime в формат даты и времени, чтобы можно было извлекать полезные признаки, такие как день недели, месяц или время года.

3. Инжиниринг признаков: Создайте новые признаки, которые могут быть полезны для модели. Например, вы можете преобразовать AgeuponOutcome в числовой формат (количество дней или лет). Также можно извлечь информацию из породы (Breed) и цвета (Color), возможно, уменьшив количество уникальных категорий или выделив основные породы и цвета.

4. Кодирование категориальных переменных: Поскольку большинство ваших столбцов являются категориальными (тип object), вам нужно будет закодировать их перед обучением модели. Вы можете использовать one-hot encoding или label encoding в зависимости от количества уникальных значений в каждом столбце.

5. Разделение данных: Разделите данные на обучающую и тестовую выборки для проверки производительности модели.

In [5]:
data.isna().sum()

AnimalID              0
Name               7691
DateTime              0
OutcomeType           0
OutcomeSubtype    13612
AnimalType            0
SexuponOutcome        1
AgeuponOutcome       18
Breed                 0
Color                 0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26729 entries, 0 to 26728
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AnimalID        26729 non-null  object
 1   Name            19038 non-null  object
 2   DateTime        26729 non-null  object
 3   OutcomeType     26729 non-null  object
 4   OutcomeSubtype  13117 non-null  object
 5   AnimalType      26729 non-null  object
 6   SexuponOutcome  26728 non-null  object
 7   AgeuponOutcome  26711 non-null  object
 8   Breed           26729 non-null  object
 9   Color           26729 non-null  object
dtypes: object(10)
memory usage: 2.0+ MB


In [7]:
data = data.drop('OutcomeSubtype', axis=1)

In [8]:
# Создание новой бинарной переменной для столбца Name
data['HasName'] = data['Name'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [9]:
data = data.drop('Name', axis=1)

In [10]:
# Заполнение пропущенных значений в SexuponOutcome и AgeuponOutcome наиболее частым значением
data['SexuponOutcome'] = data['SexuponOutcome'].fillna(data['SexuponOutcome'].mode()[0])

In [11]:
# Для преобразования возраста животных из формата строк в числовой формат (в днях), вы можете использовать следующий код:
def age_to_days(age_str):
    if pd.isnull(age_str):
        return None
    num, unit = age_str.split(' ')
    num = int(num)
    if 'year' in unit:
        return num * 365
    elif 'month' in unit:
        return num * 30
    elif 'week' in unit:
        return num * 7
    elif 'day' in unit:
        return num
    else:
        return None

# Преобразование возраста из строки в числовой формат (в днях)
data['AgeuponOutcome'] = data['AgeuponOutcome'].apply(age_to_days)

In [12]:
# заполняем пустые значения средним значением по столбцу
data['AgeuponOutcome'] = data['AgeuponOutcome'].fillna(data['AgeuponOutcome'].mean())

In [13]:
# Преобразование столбца DateTime в формат даты и времени
data['DateTime'] = pd.to_datetime(data['DateTime'])

data['DayOfWeek'] = data['DateTime'].dt.dayofweek
data['Month'] = data['DateTime'].dt.month
data['Year'] = data['DateTime'].dt.year

In [14]:
data = data.drop('DateTime', axis=1)

Также можно извлечь информацию из породы (Breed) и цвета (Color), возможно, уменьшив количество уникальных категорий или выделив основные породы и цвета

Упрощение категорий: Если в данных слишком много уникальных пород или цветов, вы можете упростить их, объединив редкие категории в общие группы. Например, все породы, которые встречаются менее чем в 1% случаев, можно объединить в категорию "Other".

In [15]:
# Упрощение категорий пород
breed_counts = data['Breed'].value_counts()
rare_breeds = breed_counts[breed_counts < data.shape[0] * 0.01].index   # 1% от общего количества животных
data['SimplifiedBreed'] = data['Breed'].apply(lambda x: 'Other' if x in rare_breeds else x)

# Упрощение категорий цветов
color_counts = data['Color'].value_counts()
rare_colors = color_counts[color_counts < data.shape[0] * 0.01].index  # 1% от общего количества животных
data['SimplifiedColor'] = data['Color'].apply(lambda x: 'Other' if x in rare_colors else x)

In [16]:
data

,AnimalID,OutcomeType,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,HasName,DayOfWeek,Month,Year,SimplifiedBreed,SimplifiedColor
0,A671945,Return_to_owner,Dog,Neutered Male,365.0,Shetland Sheepdog Mix,Brown/White,1,2,2,2014,Other,Brown/White
1,A656520,Euthanasia,Cat,Spayed Female,365.0,Domestic Shorthair Mix,Cream Tabby,1,6,10,2013,Domestic Shorthair Mix,Other
2,A686464,Adoption,Dog,Neutered Male,730.0,Pit Bull Mix,Blue/White,1,5,1,2015,Pit Bull Mix,Blue/White
3,A683430,Transfer,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Cream,0,4,7,2014,Domestic Shorthair Mix,Other
4,A667013,Transfer,Dog,Neutered Male,730.0,Lhasa Apso/Miniature Poodle,Tan,0,4,11,2013,Other,Tan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26724,A702446,Transfer,Cat,Intact Male,30.0,Domestic Shorthair Mix,Brown Tabby/White,0,3,5,2015,Domestic Shorthair Mix,Brown Tabby/White
26725,A718934,Transfer,Cat,Spayed Female,90.0,Domestic Shorthair Mix,Brown Tabby,0,2,1,2016,Domestic Shorthair Mix,Brown Tabby
26726,A698128,Adoption,Dog,Neutered Male,1460.0,Old English Bulldog Mix,White/Tan,1,0,3,2015,Other,White/Tan
26727,A677478,Transfer,Cat,Intact Male,28.0,Domestic Shorthair Mix,Black,0,6,4,2014,Domestic Shorthair Mix,Black


In [17]:
data = data.drop(['Breed', 'Color'], axis=1)

Кодирование категориальных переменных: Поскольку большинство ваших столбцов являются категориальными (тип object), вам нужно будет закодировать их перед обучением модели. Вы можете использовать one-hot encoding или label encoding в зависимости от количества уникальных значений в каждом столбце.

one-hot encoding - если не важен порядок
label encoding - если важен

In [18]:
# преобразуем OneHotEncoding
data = pd.get_dummies(data, columns=['SimplifiedColor', 'SimplifiedBreed', 'AnimalType', 'SexuponOutcome', 'OutcomeType'])

In [19]:
# Удаление ненужных столбцов
data = data.drop(['AnimalID'], axis=1)

In [20]:
# Преобразование типов данных
for col in data.columns:
    if data[col].dtype == 'bool':
        data[col] = data[col].astype(int)

In [21]:
# Нормализация/стандартизация числовых признаков
scaler = StandardScaler()
data['AgeuponOutcome'] = scaler.fit_transform(data[['AgeuponOutcome']])

In [22]:
# Проверка на дубликаты
data = data.drop_duplicates()

# Разделим на X_train и y_train и обработаем X_test 

In [23]:
# Отделение признаков (X) от целевой переменной (y)
X_train = data.drop(['OutcomeType_Adoption', 'OutcomeType_Died', 'OutcomeType_Euthanasia', 'OutcomeType_Return_to_owner', 'OutcomeType_Transfer'], axis=1)
y_train = data[['OutcomeType_Adoption', 'OutcomeType_Died', 'OutcomeType_Euthanasia', 'OutcomeType_Return_to_owner', 'OutcomeType_Transfer']]

In [24]:
X_test = pd.read_csv('csv/animal_test.csv', index_col='ID')

In [25]:
X_test

,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
ID,,,,,,,
1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White
...,...,...,...,...,...,...,...
11452,NaN,2014-07-08 14:50:00,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
11453,NaN,2014-10-21 12:57:00,Cat,Intact Female,2 weeks,Domestic Shorthair Mix,Blue
11454,NaN,2014-09-29 09:00:00,Cat,Intact Female,1 year,Domestic Shorthair Mix,Calico


In [26]:
X_test.isna().sum()

Name              3225
DateTime             0
AnimalType           0
SexuponOutcome       0
AgeuponOutcome       6
Breed                0
Color                0
dtype: int64

In [27]:
# Создание новой бинарной переменной для столбца Name
X_test['HasName'] = X_test['Name'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [28]:
X_test = X_test.drop('Name', axis=1)

In [29]:
# Для преобразования возраста животных из формата строк в числовой формат (в днях), вы можете использовать следующий код:
def age_to_days(age_str):
    if pd.isnull(age_str):
        return None
    num, unit = age_str.split(' ')
    num = int(num)
    if 'year' in unit:
        return num * 365
    elif 'month' in unit:
        return num * 30
    elif 'week' in unit:
        return num * 7
    elif 'day' in unit:
        return num
    else:
        return None

# Преобразование возраста из строки в числовой формат (в днях)
X_test['AgeuponOutcome'] = X_test['AgeuponOutcome'].apply(age_to_days)

In [30]:
# заполняем пустые значения средним значением по столбцу
X_test['AgeuponOutcome'] = X_test['AgeuponOutcome'].fillna(X_test['AgeuponOutcome'].mean())

In [31]:
# Преобразование столбца DateTime в формат даты и времени
X_test['DateTime'] = pd.to_datetime(X_test['DateTime'])

X_test['DayOfWeek'] = X_test['DateTime'].dt.dayofweek
X_test['Month'] = X_test['DateTime'].dt.month
X_test['Year'] = X_test['DateTime'].dt.year

In [32]:
X_test = X_test.drop('DateTime', axis=1)

In [33]:
# Упрощение категорий пород
breed_counts = X_test['Breed'].value_counts()
rare_breeds = breed_counts[breed_counts < X_test.shape[0] * 0.01].index   # 1% от общего количества животных
X_test['SimplifiedBreed'] = X_test['Breed'].apply(lambda x: 'Other' if x in rare_breeds else x)

# Упрощение категорий цветов
color_counts = X_test['Color'].value_counts()
rare_colors = color_counts[color_counts < X_test.shape[0] * 0.01].index  # 1% от общего количества животных
X_test['SimplifiedColor'] = X_test['Color'].apply(lambda x: 'Other' if x in rare_colors else x)

In [34]:
X_test = X_test.drop(['Breed', 'Color'], axis=1)

In [35]:
# преобразуем OneHotEncoding
X_test = pd.get_dummies(X_test, columns=['SimplifiedColor', 'SimplifiedBreed', 'AnimalType', 'SexuponOutcome'])

In [36]:
# Преобразование типов данных
for col in X_test.columns:
    if X_test[col].dtype == 'bool':
        X_test[col] = X_test[col].astype(int)

In [37]:
# Нормализация/стандартизация числовых признаков
scaler = StandardScaler()
X_test['AgeuponOutcome'] = scaler.fit_transform(X_test[['AgeuponOutcome']])

In [38]:
# # Выравнивание столбцов X_train и X_test
# X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

In [39]:
# в тестовой на один признак столбец больше, удаление столбца SimplifiedColor_Brown Brindle из тестовой выборки
X_test.drop(['SimplifiedColor_Brown Brindle'], axis=1, inplace=True)

Возникла ошибка, потому что predict_proba возвращает массив формы (n_samples, n_classes), где n_samples - это количество примеров в тестовой выборке, а n_classes - это количество классов. Однако, судя по ошибке, ваш массив имеет три измерения вместо двух.

Это может произойти, если y_train был закодирован с помощью one-hot encoding, что приводит к тому, что RandomForestClassifier рассматривает вашу задачу как несколько независимых бинарных классификаций, а не как одну многоклассовую классификацию.

Чтобы решить эту проблему, вам нужно использовать LabelEncoder для кодирования y_train перед обучением модели, чтобы преобразовать метки классов из one-hot encoded формата в формат с одним столбцом:

In [40]:
from sklearn.preprocessing import LabelEncoder

# Кодирование y_train
le = LabelEncoder()
y_train = le.fit_transform(y_train.idxmax(axis=1))

# Выбор модели | обучение | оценка

#### Случайный лес:

In [41]:
from sklearn.model_selection import GridSearchCV

# Определение параметров для GridSearchCV
param_grid_rf = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [10, 20, 30],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4]
}

# Создание пайплайна
pipeline_rf = Pipeline([
    ('randomforestclassifier', RandomForestClassifier(random_state=42))
])

# Поиск лучших параметров с помощью GridSearchCV
grid_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)

# Вывод лучших параметров и лучшего счета
print('Лучшие параметры для случайного леса:', grid_rf.best_params_)
print('Лучший счет для случайного леса:', grid_rf.best_score_)

Лучшие параметры для случайного леса: {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 10, 'randomforestclassifier__n_estimators': 100}
Лучший счет для случайного леса: 0.6373241106454264


#### Градиентный бустинг:

In [42]:
from xgboost import XGBClassifier

# Определение параметров для GridSearchCV
param_grid_xgb = {
    'xgbclassifier__n_estimators': [100, 200, 300],
    'xgbclassifier__max_depth': [3, 6, 9],
    'xgbclassifier__learning_rate': [0.01, 0.1, 0.3],
    'xgbclassifier__subsample': [0.8, 0.9, 1.0]
}

# Создание пайплайна
pipeline_xgb = Pipeline([
    ('xgbclassifier', XGBClassifier(random_state=42, n_jobs=-1))
])

# Поиск лучших параметров с помощью GridSearchCV
grid_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, scoring='accuracy')
grid_xgb.fit(X_train, y_train)

# Вывод лучших параметров и лучшего счета
print('Лучшие параметры для XGBoost:', grid_xgb.best_params_)
print('Лучший счет для XGBoost:', grid_xgb.best_score_)

Лучшие параметры для XGBoost: {'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__max_depth': 6, 'xgbclassifier__n_estimators': 100, 'xgbclassifier__subsample': 1.0}
Лучший счет для XGBoost: 0.6443155598533744


#### Предсказание Тестовой выборки:

In [45]:
# Сделаем предсказания для тестовой выборки
y_pred_proba = grid_xgb.predict_proba(X_test)

# Подготовим файл для отправки на Kaggle
submission = pd.DataFrame(y_pred_proba, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
submission['ID'] = X_test.index
submission.to_csv('sample_submission1.csv', index=False)


Ваш результат 0.83606 означает, что вы достигли довольно высокой точности в предсказании вероятностей исходов для животных, и это отличный результат, особенно если учесть, что это ваше первое участие в таком соревновании. Поздравляю с таким успехом! Если у вас есть еще вопросы или если вы хотите обсудить способы улучшения вашей модели, не стесняйтесь спрашивать.


Привет, я начинающий специалист по МЛ. Собираюсь справится с тренировочным соревнованием на Kaggle. Вот описание на англ. ниже. А также тренировочный датасет я прикрепил.

Description
Every year, approximately 7.6 million companion animals end up in US shelters. Many animals are given up as unwanted by their owners, while others are picked up after getting lost or taken out of cruelty situations. Many of these animals find forever families to take them home, but just as many are not so lucky. 2.7 million dogs and cats are euthanized in the US every year.



Using a dataset of intake information including breed, color, sex, and age from the Austin Animal Center, we're asking Kagglers to predict the outcome for each animal.

We also believe this dataset can help us understand trends in animal outcomes. These insights could help shelters focus their energy on specific animals who need a little extra help finding a new home. We encourage you to publish your insights on Scripts so they are publicly accessible.

Acknowledgements
Kaggle is hosting this competition for the machine learning community to use for data science practice and social good. The dataset is brought to you by Austin Animal Center. Shelter animal statistics were taken from the ASPCA.

Glamour shots of Kaggle's shelter pets are pictured above. From left to right: Shelby, Bailey, Hazel, Daisy, and Yeti.

Evaluation
Submissions are evaluated using the multi-class logarithmic loss. Each incident has been labeled with one true class. For each animal, you must submit a set of predicted probabilities (one for every class). The formula is then,


where N is the number of animals in the test set, M is the number of outcomes, \\(log\\) is the natural logarithm, \\(y_{ij}\\) is 1 if observation \\(i\\) is in outcome \\(j\\) and 0 otherwise, and \\(p_{ij}\\) is the predicted probability that observation \\(i\\) belongs to outcome \\(j\\).

The submitted probabilities for a given animal are not required to sum to one because they are rescaled prior to being scored (each row is divided by the row sum). In order to avoid the extremes of the log function, predicted probabilities are replaced with \\(max(min(p,1-10^{-15}),10^{-15})\\).

Submission Format
You must submit a csv file with the animal id, all candidate outcome names, and a probability for each outcome. The order of the rows does not matter. The file must have a header and should look like the following:

AnimalID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
A715022,1,0,0,0,0
A677429,0.5,0.3,0.2,0,0
...
etc.

помоги мне справиться с этим соревнованием. Изучи то что я тебе отправил, вот эти шаги я уже сделал:

1. Изучение данных: Загрузите и изучите данные, чтобы понять структуру и содержание. Посмотрите на распределение целевой переменной (исход) и проверьте наличие пропущенных значений.

2. Предварительная обработка данных: Подготовьте данные для моделирования. Это может включать заполнение пропущенных значений, кодирование категориальных переменных и нормализацию числовых переменных.

3. Разделение данных: Разделите данные на обучающую и тестовую выборки. Это позволит вам оценить производительность модели на независимых данных.

дальше мне необходимо выбрать 2 модели случайный лес и градиентный бустинг и помоги мне используя кросс валидацию, метод pipeline перебрать параметры grid_params, также используя gridsearch - найти лучшие параметры и на них сделать предсказание X_test в конце выгрузить результат в виде:

submission = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
submission['ID'] = X_test.index  # Добавляем ID животных
submission.to_csv('sample_submission.csv', index=False)

согласно заданию:
The submitted probabilities for a given animal are not required to sum to one because they are rescaled prior to being scored (each row is divided by the row sum). In order to avoid the extremes of the log function, predicted probabilities are replaced with \\(max(min(p,1-10^{-15}),10^{-15})\\).

Submission Format
You must submit a csv file with the animal id, all candidate outcome names, and a probability for each outcome. The order of the rows does not matter. The file must have a header and should look like the following:

AnimalID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
A715022,1,0,0,0,0
A677429,0.5,0.3,0.2,0,0

прочитай это и напиги мне что все изучил, после чего я сброшу тебе первую половину - код который я написал сам, а ты уже продолжишь
думай хорошо, шаг за шагом, это важно для моей карьеры, я дам тебе чаевые 20$ 